# User Behavior Overview

## Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_selector import DfSelector
from vis_plotly import *

## Data reading

In [5]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146882 entries, 0 to 146881
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   bearer_id                       146882 non-null  int64  
 1   start                           146882 non-null  object 
 2   start_ms                        146882 non-null  float64
 3   end                             146882 non-null  object 
 4   end_ms                          146882 non-null  float64
 5   dur_(ms)                        146882 non-null  float64
 6   imsi                            146882 non-null  int64  
 7   msisdn_number                   146882 non-null  int64  
 8   imei                            146882 non-null  int64  
 9   last_location_name              146882 non-null  object 
 10  avg_rtt_dl_(ms)                 146882 non-null  float64
 11  avg_rtt_ul_(ms)                 146882 non-null  float64
 12  avg_bearer_tp_dl

##  Top 10 handsets used by the customers

In [6]:
selector = DfSelector()

In [7]:
count_df = selector.filter_by_count(df, 'handset_type')
count_df.head(10)

,handset_type,count
0,Huawei B528S-23A,28190
1,Apple iPhone 6S (A1688),9369
2,Apple iPhone 6 (A1586),8967
3,Apple iPhone 7 (A1778),6240
4,Apple iPhone Se (A1723),5151
5,Apple iPhone 8 (A1905),4961
6,Apple iPhone Xr (A2105),4542
7,Samsung Galaxy S8 (Sm-G950F),4404
8,Apple iPhone X (A1901),3788
9,Samsung Galaxy A5 Sm-A520F,3672


In [8]:
fig = px.histogram(count_df.nlargest(10, 'count'), x="handset_type", y='count')
fig.show()

## Top 3 handset manufacturers

In [9]:
top_manufacturers = selector.filter_by_count(df, 'handset_manufacturer')
top_manufacturers.head(3)

,handset_manufacturer,count
0,Apple,67889
1,Samsung,39897
2,Huawei,33754


##  Top 5 handsets per top 3 handset manufacturer

In [10]:
top3_manufacturers = top_manufacturers.head(3)['handset_manufacturer'].values
top_manufacturers = df[df["handset_manufacturer"].isin(top3_manufacturers)]
top_manufacturers['handset_type'].groupby(
    df['handset_manufacturer']).apply(lambda x: x.value_counts().head(5))

handset_manufacturer                                
Apple                 Apple iPhone 6S (A1688)            9369
                      Apple iPhone 6 (A1586)             8967
                      Huawei B528S-23A                   8817
                      Apple iPhone 7 (A1778)             6240
                      Apple iPhone Se (A1723)            5151
Huawei                Huawei B528S-23A                  19373
                      Huawei E5180                       2062
                      Huawei P20 Lite Huawei Nova 3E     2000
                      Huawei P20                         1471
                      Huawei Y6 2018                      984
Samsung               Samsung Galaxy S8 (Sm-G950F)       4404
                      Samsung Galaxy A5 Sm-A520F         3672
                      Samsung Galaxy J5 (Sm-J530)        3663
                      Samsung Galaxy J3 (Sm-J330)        3431
                      Samsung Galaxy S7 (Sm-G930X)       3106
Name: handset_typ

# Task 1.1

## Number of xDR sessions

In [11]:
sessions = selector.find_agg(df, 'msisdn_number', 'count', 'bearer_id', 10, False)
sessions

,msisdn_number,bearer_id
13381,33626320676,18
6353,33614892860,17
13037,33625779332,17
36669,33659725664,16
91945,33760536639,15
75563,33675877202,15
64471,33667163239,13
13845,33627080969,12
91604,33760413819,12
1258,33604515716,12


## Session duration

In [12]:
duration = df.groupby('msisdn_number').agg({'dur_(ms)': 'sum'})
duration = duration.rename(columns={'dur_(ms)': 'total_duration(ms)'})
duration.sort_values(by=['total_duration(ms)'], ascending=False).head(10)

,total_duration(ms)
msisdn_number,
33625779332,4137804.00
33626320676,4006827.00
33614892860,3899805.00
33659725664,3492198.00
33760536639,3396476.00
33675877202,3331851.00
33667163239,3113767.00
33603127838,2855128.00
33627080969,2782060.00


In [13]:
duration.describe()

,total_duration(ms)
count,105714.00
mean,139190.15
std,129395.17
min,7142.00
25%,72723.50
50%,103107.00
75%,172799.00
max,4137804.00


## The total download (DL) and upload (UL) data

In [14]:
usage = df.groupby('msisdn_number')[['total_dl_(bytes)', 'total_ul_(bytes)']].sum()
usage['total_data'] = usage['total_dl_(bytes)'] + usage['total_ul_(bytes)']
usage.nlargest(20, 'total_data')

,total_dl_(bytes),total_ul_(bytes),total_data
msisdn_number,,,
33614892860,8156743493.00,689483001.00,8846226494.00
33760536639,7811295382.00,703478581.00,8514773963.00
33625779332,7770043342.00,729577380.00,8499620722.00
33626320676,7301516540.00,669650721.00,7971167261.00
33675877202,7309541816.00,581568792.00,7891110608.00
33659725664,7081602462.00,624260321.00,7705862783.00
33666464084,6903439962.00,405060976.00,7308500938.00
33760413819,6610851624.00,521518890.00,7132370514.00
33664712899,6400773755.00,471562499.00,6872336254.00


In [15]:
usage.describe()

,total_dl_(bytes),total_ul_(bytes),total_data
count,105714.00,105714.00,105714.00
mean,631711503.63,57135093.16,688846596.79
std,460920510.16,35268950.20,486996120.50
min,8827082.00,9493162.00,33249009.00
25%,314222219.75,36375624.75,357909940.50
50%,569321331.50,46763051.00,616921019.00
75%,806265196.50,65502749.00,856440351.75
max,8156743493.00,729577380.00,8846226494.00


## The total data volume (in Bytes) during this session for each application

In [16]:
df["social_media"] = df["social_media_dl_(bytes)"] + df['social_media_ul_(bytes)']
df["google"] = df["google_dl_(bytes)"] + df["google_ul_(bytes)"]
df['email'] = df["email_dl_(bytes)"] + df["email_ul_(bytes)"]
df['youtube'] = df["youtube_dl_(bytes)"] + df["youtube_ul_(bytes)"]
df['netflix'] = df["netflix_dl_(bytes)"] + df["netflix_ul_(bytes)"]
df["gaming"] = df["gaming_dl_(bytes)"] + df["gaming_ul_(bytes)"]
df['other'] = df["other_dl_(bytes)"]+df["other_ul_(bytes)"]
df['total_data'] = df['total_dl_(bytes)'] + df['total_ul_(bytes)']

In [17]:
apps_df = df.groupby('msisdn_number')[['social_media', 'google', 'email', 'youtube', 'gaming', 'other', 'total_data']].sum().nlargest(20, 'total_data')
apps_df

,social_media,google,email,youtube,gaming,other,total_data
msisdn_number,,,,,,,
33614892860,28294544.00,127973787.00,40788634.00,394370218.00,7749432234.00,7639263572.00,8846226494.00
33760536639,39783189.00,123223099.00,33693767.00,396289198.00,7461045228.00,4716134493.00,8514773963.00
33625779332,27135500.00,142307915.00,40633966.00,452958769.00,7326673487.00,6354583086.00,8499620722.00
33626320676,43374779.00,152191852.00,42418782.00,374483047.00,6887572116.00,8167877776.00,7971167261.00
33675877202,19222921.00,109860502.00,31514421.00,317410572.00,6970567597.00,6798515150.00,7891110608.00
33659725664,35412358.00,116516345.00,35999792.00,257991088.00,6725559211.00,6317415487.00,7705862783.00
33666464084,18629986.00,89320737.00,25557139.00,227336012.00,6646303338.00,4349141478.00,7308500938.00
33760413819,20777205.00,82738720.00,25576965.00,303169107.00,6268619592.00,4101645436.00,7132370514.00
33664712899,19390599.00,90389372.00,21426007.00,276834013.00,6103856008.00,3976960308.00,6872336254.00


In [20]:
sort_by_total_data = apps_df.sort_values(
    'total_data', ascending=False)['total_data']
hist(sort_by_total_data)

In [ ]:
sorted_by_tp = user_experience_df.sort_values(
    'total_avg_tp', ascending=False)
top_10 = sorted_by_tp.head(10)['total_avg_tp']
last_10 = sorted_by_tp.tail(10)['total_avg_tp']
most_10 = user_engagement_df['total_avg_tp'].value_counts().head(10) mult_hist([top_10, last_10, most_10, top_10], 1,
                                                                               3, "TCP values in the dataset", ['Top 10', 'Last 10', 'Most 10'])


In [ ]:
df.to_csv('../data/clean_data.csv', index=False)